In [1]:
import csv
import json
import argparse
import os
os.environ["CUDA_VISIBLE_DEVICES"]='2, 7'
import torch
import random
import transformers
import time
import re
from vllm import LLM, SamplingParams
from tqdm import tqdm
import logging
import sys
from datasets import load_dataset
import pandas as pd
class args:
    model = "/data/naifanzhang/model/LLM-Research/Phi-3.5-MoE-instruct"
    gpu_util = 0.9
    moe_path="/data/naifanzhang/exp/moe_drop/expert_cnt_class1_remove0.pth"


choices = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P"]
max_model_length = 4096
max_new_tokens = 2048

/home/naifanzhang/.conda/envs/eedi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-23 02:23:19,254	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
torch.load("/data/naifanzhang/moe_purning/mmlu/phi_mask/final/computer_science_ours16_global.pth")

/tmp/ipykernel_1812333/2447363447.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load("/data/naifanzhang/moe_purning/mmlu/phi_mask/final/computer_science_ours16_g

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1,

In [ ]:
model="/data/naifanzhang/model/LLM-Research/Phi-3.5-MoE-instruct"
llm = LLM(model=model, gpu_memory_utilization=float(args.gpu_util),
                tensor_parallel_size=torch.cuda.device_count(),
                max_model_len=max_model_length,
                trust_remote_code=True)

In [4]:
base_model = llm.llm_engine.model_executor.driver_worker.model_runner.model

In [9]:
# 统计有多少层包含block_sparse_moe
moe_layer_count = 0
for layer in base_model.model.layers:
    if hasattr(layer, 'block_sparse_moe'):
        moe_layer_count += 1
print(f"模型中共有 {moe_layer_count} 层包含block_sparse_moe")

模型中共有 32 层包含block_sparse_moe


In [15]:
torch.load(args.moe_path).shape

/tmp/ipykernel_2579352/170493118.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(args.moe_path).shape


torch.Size([32, 8])

In [11]:
all_ont_mask = torch.ones([32,16])
# 保存为pth文件
torch.save(all_ont_mask, "/data/naifanzhang/exp/moe_drop/all_one_mask_phi.pth")


In [9]:
all_ont_mask = torch.zeros([32,16])
all_ont_mask[:,0] = 1
all_ont_mask[:,1] = 1
# all_ont_mask[:,0] = 0
# all_ont_mask[:,0] = 0
# all_ont_mask[:,0] = 0
torch.save(all_ont_mask, "/data/naifanzhang/exp/moe_drop/all_one_mask_phi.pth")

In [12]:
torch.load("/data/naifanzhang/exp/moe_drop/all_one_mask_phi.pth")

/tmp/ipykernel_2634403/854855094.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load("/data/naifanzhang/exp/moe_drop/all_one_mask_phi.pth")


tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1

In [3]:
from types import MethodType
from typing import Optional
import torch.nn.functional as F

def new_forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
    # NOTE: hidden_states can have either 1D or 2D shape.
    layer_expert_mask = getattr(self, 'current_expert_masks', None)
    print(f"layer_expert_mask is {layer_expert_mask}")
    orig_shape = hidden_states.shape
    hidden_states = hidden_states.view(-1, self.hidden_size)
    # router_logits: (num_tokens, n_experts)
    router_logits, _ = self.gate(hidden_states)
    print(f"router_logits is {router_logits}")
    print(f"router_logits shape is {router_logits.shape}")
    if layer_expert_mask is not None:
        # 创建与router_logits相同shape的mask
        expert_mask = torch.zeros_like(router_logits, dtype=torch.bool)
        # 根据layer_expert_mask设置允许的专家
        expert_mask[:, layer_expert_mask] = True
        router_logits = router_logits.masked_fill(~expert_mask, float('-inf'))
    print(f"After mask router_logits is {router_logits}")
    print(f"After mask router_logits shape is {router_logits.shape}")

    final_hidden_states = self.experts(hidden_states, router_logits)
    return final_hidden_states.view(orig_shape)

def forward_with_expert_masks(self, *args, **kwargs):
    """
    Args:
        expert_masks: List[torch.Tensor]，每个元素对应一个MoE层的mask
    """
    # 从LLM实例获取expert_masks
    # 这里的self指代的是MixtralForCausalLM模型实例
    # 因为forward_with_expert_masks方法是通过MethodType绑定到MixtralForCausalLM模型上的
    # 可以通过打印self的类型来验证
    print(f"self type in forward_with_expert_masks is: {type(self)}")
    expert_masks = getattr(self, 'current_expert_masks', None)
    print(f"forward_with_expert_masks in self.current_expert_masks is {expert_masks}")
            
    moe_layer_idx = 0
    for idx, layer in enumerate(self.model.layers):
        if hasattr(layer, 'block_sparse_moe'):
            print(f"layer type in forward_with_expert_masks is: {type(layer)}")
            # 为每个MoE层设置对应的expert_mask
            layer.block_sparse_moe.current_expert_masks = expert_masks[moe_layer_idx] if expert_masks is not None else None
            #print(f"Moe block {idx} mask is: {layer.block_sparse_moe.current_expert_mask}")
            moe_layer_idx += 1
    
    outputs = self.original_forward(*args, **kwargs)
    return outputs

def modify_moe_layers(model):
    if hasattr(model, 'model_runner'):
        model.model_runner = model.model_runner
    print(f"--------------------------------")
    print(f"In Moe Model Top current_expert_masks is {model.current_expert_masks}")

    # 修改每个MoE层
    for layer in model.model.layers:
        if hasattr(layer, 'block_sparse_moe'):
            moe_layer = layer.block_sparse_moe
            moe_layer.forward = MethodType(new_forward, moe_layer)
    
    model.original_forward = model.forward
    model.forward = MethodType(forward_with_expert_masks, model)

class ExpertMaskLLM(LLM):
    """扩展LLM类以支持expert masks"""
    def __init__(self, expert_masks=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        #self.current_expert_masks = None
        self.llm_engine.model_executor.driver_worker.model_runner.model.current_expert_masks = expert_masks
        self.current_expert_masks = expert_masks
        print(self.llm_engine.model_executor.driver_worker.model_runner.model)
        modify_moe_layers(self.llm_engine.model_executor.driver_worker.model_runner.model)

In [4]:
def load_expert_masks(moe_path):
    expert_mask= torch.load(moe_path)
    num_layers = len(expert_mask)
    expert_masks = []
    for layer_idx in range(num_layers):
        active_experts = torch.nonzero(expert_mask[layer_idx]).squeeze()
        expert_masks.append(active_experts)
    return expert_masks

In [7]:
expert_masks = load_expert_masks(args.moe_path)

/tmp/ipykernel_2569301/3556500580.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  expert_mask= torch.load(moe_path)


In [ ]:
sampling_params = SamplingParams(temperature=0, max_tokens=max_new_tokens,
                                    stop=["Question:"])

moe_path= "/data/naifanzhang/exp/moe_drop/expert_cnt_class1_remove36.pth"
expert_mask= torch.load(moe_path)
num_layers = len(expert_mask)
expert_masks = []
for layer_idx in range(num_layers):
    active_experts = torch.nonzero(expert_mask[layer_idx]).squeeze()
    expert_masks.append(active_experts)

model="/data/naifanzhang/model/LLM-Research/Phi-3.5-MoE-instruct"
llm = ExpertMaskLLM(model=model, gpu_memory_utilization=float(args.gpu_util),
                tensor_parallel_size=torch.cuda.device_count(),
                max_model_len=max_model_length,
                trust_remote_code=True,
                expert_masks=expert_masks)


In [8]:
model="/data/naifanzhang/model/LLM-Research/Phi-3.5-MoE-instruct"
llm = ExpertMaskLLM(model=model, gpu_memory_utilization=float(args.gpu_util),
                tensor_parallel_size=torch.cuda.device_count(),
                max_model_len=max_model_length,
                trust_remote_code=True,
                expert_masks=expert_masks)

INFO 11-17 14:48:00 config.py:107] Replacing legacy 'type' key with 'rope_type'


INFO 11-17 14:48:05 config.py:905] Defaulting to use mp for distributed inference
INFO 11-17 14:48:05 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/data/naifanzhang/model/LLM-Research/Phi-3.5-MoE-instruct', speculative_config=None, tokenizer='/data/naifanzhang/model/LLM-Research/Phi-3.5-MoE-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False

(VllmWorkerProcess pid=2570813) /home/naifanzhang/.conda/envs/eedi/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
(VllmWorkerProcess pid=2570813)   @torch.library.impl_abstract("xformers_flash::flash_fwd")
(VllmWorkerProcess pid=2570813) /home/naifanzhang/.conda/envs/eedi/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/naifanzhang/.conda/envs/eedi/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that ins

(VllmWorkerProcess pid=2570813) INFO 11-17 14:48:21 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 11-17 14:48:26 utils.py:1008] Found nccl from library libnccl.so.2
INFO 11-17 14:48:26 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=2570813) INFO 11-17 14:48:26 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=2570813) INFO 11-17 14:48:26 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 11-17 14:48:26 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /home/naifanzhang/.cache/vllm/gpu_p2p_access_cache_for_2, 4.json
(VllmWorkerProcess pid=2570813) INFO 11-17 14:48:26 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /home/naifanzhang/.cache/vllm/gpu_p2p_access_cache_for_2, 4.json
INFO 11-17 14:48:26 shm_broadcast.py:241] vLLM message queue communication handle: Handle(connect_ip='127.0.0.1', local_reader_ranks=[1], buffer=<vllm.distributed.device_communicators.shm_broadcast.ShmRingBuffer object at 0x

Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]


(VllmWorkerProcess pid=2570813) ERROR 11-17 14:48:27 multiproc_worker_utils.py:229] Exception in worker VllmWorkerProcess while processing method load_model.
(VllmWorkerProcess pid=2570813) ERROR 11-17 14:48:27 multiproc_worker_utils.py:229] Traceback (most recent call last):
(VllmWorkerProcess pid=2570813) ERROR 11-17 14:48:27 multiproc_worker_utils.py:229]   File "/home/naifanzhang/.conda/envs/eedi/lib/python3.10/site-packages/vllm/executor/multiproc_worker_utils.py", line 223, in _run_worker_process
(VllmWorkerProcess pid=2570813) ERROR 11-17 14:48:27 multiproc_worker_utils.py:229]     output = executor(*args, **kwargs)
(VllmWorkerProcess pid=2570813) ERROR 11-17 14:48:27 multiproc_worker_utils.py:229]   File "/home/naifanzhang/.conda/envs/eedi/lib/python3.10/site-packages/vllm/worker/worker.py", line 183, in load_model
(VllmWorkerProcess pid=2570813) ERROR 11-17 14:48:27 multiproc_worker_utils.py:229]     self.model_runner.load_model()
(VllmWorkerProcess pid=2570813) ERROR 11-17 14

Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:00<00:13,  1.17it/s]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:01<00:12,  1.20it/s]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:02<00:12,  1.16it/s]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:03<00:11,  1.14it/s]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:04<00:09,  1.21it/s]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:05<00:09,  1.16it/s]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:06<00:08,  1.14it/s]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:06<00:07,  1.18it/s]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:07<00:07,  1.14it/s]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:08<00:06,  1.08it/s]
Loading safetensors checkpoint shards:  65% Completed | 11/17 [00:09<00:05,  1.06it/s]
Loading safetensors checkpoint shards:  71% Complete

INFO 11-17 14:48:43 model_runner.py:1067] Loading model weights took 39.0659 GB


OutOfMemoryError: CUDA out of memory. Tried to allocate 800.00 MiB. GPU 1 has a total capacity of 79.15 GiB of which 664.12 MiB is free. Process 2419576 has 75.16 GiB memory in use. Including non-PyTorch memory, this process has 3.31 GiB memory in use. Of the allocated memory 2.67 GiB is allocated by PyTorch, and 31.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
inference_batch = ["What is the capital of France?", "What is the capital of China?"]
llm.generate(inference_batch, sampling_params)

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

self type in forward_with_expert_masks is: <class 'vllm.model_executor.models.mixtral.MixtralForCausalLM'>
forward_with_expert_masks in self.current_expert_masks is [tensor([0, 1, 2, 3, 4]), tensor([1, 5, 6, 7]), tensor([1, 2, 3, 4, 6, 7]), tensor([1, 3, 4, 5, 6, 7]), tensor([0, 1, 4, 6, 7]), tensor([0, 1, 3, 4, 5, 7]), tensor([1, 5, 6, 7]), tensor([2, 3, 4, 6]), tensor([0, 1, 2, 5, 6]), tensor([1, 2, 5, 6, 7]), tensor([0, 1, 2, 3, 4, 5]), tensor([0, 1, 4, 5, 6]), tensor([2, 3, 4, 5, 6]), tensor([1, 4, 5, 6, 7]), tensor([0, 1, 3, 5, 6]), tensor([0, 4, 5, 6, 7]), tensor([1, 2, 3, 4, 5]), tensor([0, 1, 3, 4, 5, 7]), tensor([0, 2, 3, 4, 5]), tensor([2, 4, 5, 7]), tensor([0, 2, 5, 7]), tensor([1, 3, 5, 6, 7]), tensor([0, 3, 5, 6, 7]), tensor([4, 5, 6, 7]), tensor([0, 1, 3, 4, 5, 6]), tensor([0, 1, 4, 5]), tensor([1, 2, 3, 4, 5, 7]), tensor([0, 1, 2, 5, 7]), tensor([1, 3, 4, 5, 7]), tensor([0, 1, 2, 4, 5, 6, 7]), tensor([0, 1, 2, 5, 6, 7]), tensor([0, 1, 2, 3, 4])]
layer type in forward_wit

Processed prompts: 100%|██████████| 2/2 [00:26<00:00, 13.27s/it, est. speed input: 0.60 toks/s, output: 78.68 toks/s]


[RequestOutput(request_id=0, prompt='What is the capital of France?', prompt_token_ids=[1, 1824, 349, 272, 5565, 302, 4843, 28804], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\n\nParis is the capital of France.\n\nWhat is the population of France?\n\nThe population of France is 66,991,000.\n\nWhat is the area of France?\n\nThe area of France is 248,573 square miles.\n\nWhat is the official language of France?\n\nThe official language of France is French.\n\nWhat is the currency of France?\n\nThe currency of France is the euro.\n\nHow did France get its name?\n\nThe name France comes from the Latin word Francia, which means “land of the Franks.”\n\nWhat is the history of France?\n\nFrance has a long and complex history. It was originally inhabited by the Gauls, who were conquered by the Romans in the 1st century BC. The Franks, a Germanic tribe, invaded the country in the 5th century AD and established the Frankish 

In [5]:
# 查看generate函数的源码
import inspect
print(inspect.getsource(llm.generate))

<bound method LLM.generate of <vllm.entrypoints.llm.LLM object at 0x785b400ee470>>

In [8]:
base_model = llm.llm_engine.model_executor.driver_worker.model_runner.model

In [9]:
base_model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): VocabParallelEmbedding(num_embeddings=16000, embedding_dim=4096, org_vocab_size=32000, num_embeddings_padded=32000, tp_size=2)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (qkv_proj): QKVParallelLinear(in_features=4096, output_features=3072, bias=False, tp_size=2, gather_output=False)
          (o_proj): RowParallelLinear(input_features=2048, output_features=4096, bias=False, tp_size=2, reduce_results=True)
          (rotary_emb): RotaryEmbedding(head_size=128, rotary_dim=128, max_position_embeddings=32768, base=1000000, is_neox_style=True)
          (attn): Attention(head_size=128, num_heads=16, num_kv_heads=4, scale=0.08838834764831845, backend=FlashAttentionImpl)
        )
        (block_sparse_moe): MixtralMoE(
          (gate): ReplicatedLinear(in_features=4096, output_features=8, bias=False)
          (experts): FusedMoE(
            (quant_method